INTEGRATION APIs

Pour l’analyse de données, il faut : 
* accéder aux données 
   * voir le type de base de données proposée par l’équipe Questionnaire (Google sheet via Tally? )
   * et confirmer l’accès (démontrer notre capacité à se connecter)
* pouvoir enrichir les données (via TMDB par ex)
   * démontrer notre capacité à récupérer des données tierces via API
   * sauf si cette étape est faite par l’équipe Questionnaire (qui veut aussi accéder aux données des films ? 
* pipeline de traitement des données pour produire les KPI 
   * décider du stockage des KPIs ou les calculer à la demande ? 
      * rester sur G-sheet ? 
      * base de données classique (postgresql, sqlite…?)
      * définir un modèle en étoile ? pas nécessaire? 

# IMPORTS

In [1]:
# dotenv configuration
import os
import requests
import typing as typ

from dotenv import load_dotenv
import numpy as np
import pandas as pd

# api integration google
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build, Resource
from googleapiclient.errors import HttpError

/var/folders/cw/pk20bql55690x9wk3ngfpvgm0000gn/T/ipykernel_18408/715639572.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# SCRIPTS

In [2]:
# Configuration dotenv
# Load the stored environment variables
load_dotenv()

# Get the values
dotenv_test = os.getenv("DOTENV_TEST")

print(f"DOTENV_TEST = {dotenv_test}")

DOTENV_TEST = Data4Good


## accès google sheet

In [3]:
# cf. https://developers.google.com/sheets/api/quickstart/python?hl=fr
# based on the quickstart template

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

def read_from_google_sheet(spreadsheet_id: str, range_name: str) -> None | typ.List[typ.Any]:
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  tokenPath = "../apis/google/token.json"
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists(tokenPath):
    creds = Credentials.from_authorized_user_file(tokenPath, SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "../apis/google/credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(tokenPath, "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=spreadsheet_id, range=range_name)
        .execute()
    )
    sheet_data: typ.List[typ.Any] | None = result.get("values", [])

    
    #if not sheet_data or len(sheet_data) == 0:
    #  print("No data found.")
    #  return None
    #
    #return sheet_data
    return sheet_data
  
  except HttpError as err:
    print("http error", err)


In [10]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = os.getenv("GOOGLE_SPREADSHEET_ID")
if SPREADSHEET_ID is None:
  raise TypeError("Missing spreadsheet id")
RANGE_NAME = "idForm1!A1:ZZZ"

google_data = read_from_google_sheet(spreadsheet_id=SPREADSHEET_ID, range_name=RANGE_NAME)
if not google_data or len(google_data) == 0:
  raise ValueError("no data")

data = pd.DataFrame(columns=google_data[0], data=google_data[1:])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   created_date  10 non-null     object
 1   answer_1      10 non-null     object
 2   answer_2      10 non-null     object
 3   filmId        10 non-null     object
 4   sourceFilmId  10 non-null     object
dtypes: object(5)
memory usage: 528.0+ bytes


# accès TMDB 

TMDb offers a powerful API service that is free to use as long as you properly attribute us as the source of the data and/or images you use. You can find the logos for attribution here.



### search movies

In [14]:
query = "imaginaires"
url = f"https://api.themoviedb.org/3/search/movie?query={query}&include_adult=false"
headers = {
    "accept": "application/json",
    "Authorization": f'Bearer {os.getenv("TMDB_API_READ_ACCESS_TOKEN")}'
}

response = requests.get(url, headers=headers)
results = response.json()
print("total_results", results['total_results'])
# 
[ t['title'] for t in results['results']]

total_results 13


['Heartbeats',
 'Imaginary Feasts',
 'Axel and the True Story of an Imaginary Illness',
 'Orsay (Musées imaginaires I)',
 'Quai Branly (Musées imaginaires II)',
 'Vagabonds imaginaires',
 'Films Imaginaires',
 'Le Malade imaginaire',
 'Come Away',
 'A Land Imagined',
 'The Invention of Imaginary Machines of Destruction',
 'Imaginary Flying Machines',
 'La bottega degli amici immaginari']

### get movies

In [20]:
movieId = "900"
url = f"https://api.themoviedb.org/3/movie/{movieId}"
headers = {
    "accept": "application/json",
    "Authorization": f'Bearer {os.getenv("TMDB_API_READ_ACCESS_TOKEN")}'
}

response = requests.get(url, headers=headers)

results = response.json()
print("title:", results['title'])
print("imdb id:", results['imdb_id'])

title: Bringing Up Baby
imdb id: tt0029947


In [26]:
#exemple budget
budgets = []
for movieId in data.filmId:
    print('id:', movieId)
    
    url = f"https://api.themoviedb.org/3/movie/{movieId}"
    headers = {
        "accept": "application/json",
        "Authorization": f'Bearer {os.getenv("TMDB_API_READ_ACCESS_TOKEN")}'
    }

    response = requests.get(url, headers=headers)
    
    budget = None
    if response.status_code != 200:
        print(response.status_code)
        budget = -999
    else:
        results = response.json()
        budget = results['budget']
    
    budgets.append(budget)
budgets

id: 540
id: 1490
id: 2450
404
id: 388
id: 1992
id: 110
id: 2457
id: 12356
404
id: 9991
id: 1999


[3500000, 0, -999, 45000000, 0, 0, 0, -999, 0, 1700000]